# Librerías

In [10]:
import calendar as cd
import pandas as pd
import numpy as np
import math
import psycopg2
import matplotlib.pyplot as plt
from collections import OrderedDict
import json
from datetime import datetime
import query


In [60]:
%run C:\Users\tomas.retamal\EDP_git\funciones.ipynb

# Traer datos

In [65]:
with open('credentials_postgre.json') as file:
	credentials = json.load(file)
conn_sql = psycopg2.connect(user = credentials["postgre"]['user'],
                            password = credentials["postgre"]['password'],
                            host = credentials["postgre"]['host'],
                            port = credentials["postgre"]['port'],
                            database = credentials["postgre"]['database'])
query_sshh = query.query_sshh
query_viajes = query.query_viajes

#Traemos las tablas básicas:
#Servicios históricos (SQL)
sshh = pd.read_sql(query_sshh, conn_sql)
#Viajes (SQL)
viajes = pd.read_sql(query_viajes, conn_sql)
#Transición y precio histórico (EXCEL)
transicion = pd.read_excel("TT y PH.xlsx",sheet_name = "Transición")
transicion = transicion.fillna(0)
precio_historico = pd.read_excel("TT y PH.xlsx",sheet_name = "Precio histórico")
#Renombramos algunas columnas de Transición y Precio Histórico para mayor comodidad.
precio_historico = precio_historico.rename(columns={'precio histórico_id':'precio_historico_id','fecha inicio':'fecha_inicio_precio','fecha término':'fecha_termino_precio'})
transicion = transicion.rename(columns={'id_servicio':'servicio_id'})

#Generamos tablas de interés mediante la combinación de ciertas tablas básicas:
#Tabla de homologación
homologacion = sshh.merge(transicion[["nombre_edp","codigo_comercial","servicio_id"]].reset_index(drop=True),'left','servicio_id')
#Tabla de viajes + características comerciales
viajes_tr_ph = viajes.merge(right=transicion, how='left', on='servicio_id',sort='viaje_id').merge(right=precio_historico, how='left', on='precio_historico_id',sort='viaje_id')
viajes_tr_ph = viajes_tr_ph.drop(["fecha_inicio1"],axis=1)
viajes_tr_ph["fecha_inicio"]=pd.to_datetime(
viajes_tr_ph["fecha_inicio"],
errors = 'coerce',
format = '%Y-%m-%d %H:%M:%S'
)#.dt.tz_localize(None)

viajes_tr_ph["fecha_inicio_cobro"]=pd.to_datetime(
viajes_tr_ph["fecha_inicio_cobro"],
errors = 'coerce',
format = '%Y-%m-%d %H:%M:%S'
)#.dt.tz_localize(None)
viajes_tr_ph["vigencia_cobro"]=pd.to_datetime(
viajes_tr_ph["vigencia_cobro"],
errors = 'coerce',
format = '%Y-%m-%d %H:%M:%S'
)
viajes_tr_ph["vigencia_servicio"]=pd.to_datetime(
viajes_tr_ph["vigencia_servicio"],
errors = 'coerce',
format = '%Y-%m-%d %H:%M:%S'
)

# Generar EDP

In [66]:
meses = {'01':'enero','02':'febrero','03':'marzo','04':'abril','05':'mayo','06':'junio','07':'julio','08' : 'agosto', '09' : 'septiembre', '10' : 'octubre', '11':'noviembre','12':'diciembre'}
contrato = "C CPO ATACAMA KOZAN"
inicio = "2021-11-01 00:00:00"
fin =    "2021-11-30 23:59:59"
dir = "C:/Users/tomas.retamal/EDP_Git/EDP Python/"
writer = pd.ExcelWriter(dir + contrato[2:5]+ "_EDP_" +contrato[6::]+"_"+fin[:4]+"_"+fin[5:7]+"_"+meses[fin[5:7]]+".xlsx")
comercial = generar_edp(viajes_tr_ph,contrato,inicio,fin)[0].rename(columns={'viaje_id':'tramos_ejecutados'})
operacional = generar_edp(viajes_tr_ph,contrato,inicio,fin)[1].rename(columns={'viaje_id':'tramos_ejecutados'})
viajes_edp = generar_edp(viajes_tr_ph,contrato,inicio,fin)[2].rename(columns={'viaje_id':'tramos_ejecutados'})
comercial.to_excel(writer, sheet_name="Comercial",index=False)
operacional.to_excel(writer, sheet_name="Operacional",index=False)
viajes_edp.to_excel(writer,sheet_name="Viajes",index=False)
writer.save()
writer.close()
#SODEXO NESTLÉ FALTAN FRECUENCIAS

C:\Users\TOMAS~1.RET\AppData\Local\Temp/ipykernel_9916/2763566288.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comercial["total"][i] = comercial["costo fijo flota"][i] * comercial["cantidad_vehiculos"][i] * comercial["disponibilidad"][i] + comercial["costo fijo logística"][i] * comercial["cantidad_vehiculos"][i] * comercial["disponibilidad"][i] + comercial["costo variable vuelta unitario logística"][i] * comercial["viaje_id"][i]
C:\Users\TOMAS~1.RET\AppData\Local\Temp/ipykernel_9916/2763566288.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comercial["total"][i] = comercial["costo variable vuelta unitari

In [63]:
generar_edp(viajes_tr_ph,contrato,inicio,fin)[0]

C:\Users\TOMAS~1.RET\AppData\Local\Temp/ipykernel_9916/2763566288.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comercial["total"][i] = comercial["costo fijo flota"][i] * comercial["cantidad_vehiculos"][i] * comercial["disponibilidad"][i] + comercial["costo fijo logística"][i] * comercial["cantidad_vehiculos"][i] * comercial["disponibilidad"][i] + comercial["costo variable vuelta unitario logística"][i] * comercial["viaje_id"][i]
C:\Users\TOMAS~1.RET\AppData\Local\Temp/ipykernel_9916/2763566288.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comercial["total"][i] = comercial["costo variable vuelta unitari

,nombre_edp,tipo de cobro,precio_historico_id,cantidad_vehiculos,disponibilidad,viaje_id,distancia,EDP separados,costo fijo flota,costo fijo logística,costo variable vuelta logística,costo variable vuelta unitario logística,total,periodo
0,ADICIONAL JEFE TURNO,Suma alzada (F+L) + cobro por vuelta,341,1.0,1.0,120,1378.064,No,658925.349985,2.635701e+06,15941.313898,7970.656949,4251105,Del 01 de noviembre al 30 de noviembre
1,APOYO ADMINISTRATIVO,Suma alzada (F+L) + cobro por vuelta,339,1.0,1.0,25,759.325,No,127760.267834,3.159323e+06,15890.580576,7945.290288,3485715,Del 01 de noviembre al 30 de noviembre
2,JEFE TURNO,Suma alzada (F+L) + cobro por vuelta,340,1.0,1.0,120,1153.550,No,658925.349985,2.635701e+06,15941.313898,7970.656949,4251105,Del 01 de noviembre al 30 de noviembre
3,LOS CARRERA,Cobro por E/S,337,1.0,1.0,21,354.652,No,NaN,NaN,NaN,29127.789894,611683,Del 01 de noviembre al 30 de noviembre
4,LOS CARRERA,Suma alzada (F+L) + cobro por vuelta,336,1.0,1.0,120,3025.404,No,775206.294100,3.100825e+06,29127.789894,14563.894947,5623698,Del 01 de noviembre al 30 de noviembre
5,NRO. 3,Suma alzada (F+L) + cobro por vuelta,335,1.0,1.0,43,746.440,No,823086.682853,3.292347e+06,22133.888449,11066.944224,4591312,Del 01 de noviembre al 30 de noviembre
6,PALOMAR,Cobro por E/S,334,1.0,1.0,21,494.878,No,NaN,NaN,NaN,29127.789894,611683,Del 01 de noviembre al 30 de noviembre
7,PALOMAR,Suma alzada (F+L) + cobro por vuelta,333,1.0,1.0,120,3382.659,No,823086.682853,3.292347e+06,29127.789894,14563.894947,5863100,Del 01 de noviembre al 30 de noviembre
8,RELAVE,Suma alzada (F+L) + cobro por vuelta,338,1.0,1.0,120,3380.452,No,775206.294100,3.100825e+06,22240.141739,11120.070869,5210439,Del 01 de noviembre al 30 de noviembre
9,SECTOR ALTO,Cobro por E/S,332,1.0,1.0,21,492.719,No,NaN,NaN,NaN,24682.860601,518340,Del 01 de noviembre al 30 de noviembre
